# Managing subplots
One of the main feature of the library is that the `slicer` dimension is shared across subplot grids. This is convenient when drawing different graphic visualization along common data facets.

Statsplotly provides utility methods to further tweak subplots grid.

In [ ]:
import plotly.io as pio

pio.renderers.default = "sphinx_gallery"

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots

import statsplotly

## Drawing subplots
All statsplotly plotting methods can be targeted to a `plotly.graph_objects.Figure` object created with plotly's `make_subplots` function. 

By default, subplot axes are independent :

In [ ]:
df = (
    px.data.experiment().melt(id_vars=["gender", "group"], var_name="experiment").set_index("group")
)

fig = make_subplots(cols=2, rows=2, column_widths=[0.9, 0.1])

for row, group in enumerate(df.index.unique(), 1):
    fig = statsplotly.distplot(
        fig=fig, row=row, col=1, data=df.loc[group], y="value", slicer="gender", barmode="overlay"
    )

    fig = statsplotly.catplot(
        fig=fig,
        row=row,
        col=2,
        data=df.loc[group],
        plot_type="stripplot",
        x="group",
        slicer="gender",
        y="value",
    )

fig.layout.height = 800
fig.show()

## Setting common axis limits

The `set_common_axis_limit` method of the `SubplotGridFormatter` class controls axis behaviour across the subplot grid. 

Here we set common limits and link yaxes across the grid rows (while preserving the different limits of each row) :

In [ ]:
from statsplotly.utils import SubplotGridFormatter

SubplotGridFormatter(fig=fig).set_common_axis_limit(shared_grid_axis="rows", link_axes=True)
fig.show()

## Tidying subplots
`SubplotGridFormatter` also exposes the `tidy_subplots` utility method for flexible annotation of a subplot grid. The method automatically set a shared title for linked axes  :

In [ ]:
SubplotGridFormatter(fig=fig).tidy_subplots(
    title="Effect of treatment",
    row_titles=["Control", "Treatment"],
    col_titles=["Distribution", "Raw Data"],
)
fig.show()

## Combining slicer and coloraxis across subplots

By chaining these methods, it is possible to combine and flexibly link subplots depicting different facets of a dataset. 

Here we link y axes across rows, x axes across all subplots, and set a shared coloraxis across rows : 

In [ ]:
fig = make_subplots(cols=2, rows=2, row_heights=[0.2, 0.8])
for col, group in enumerate(df.index.unique(), 1):
    fig = statsplotly.distplot(
        fig=fig,
        col=col,
        row=2,
        data=df.loc[group],
        x="value",
        slicer="gender",
        hist=False,
        kde=True,
    )

    fig = statsplotly.catplot(
        fig=fig,
        col=col,
        row=1,
        plot_type="stripplot",
        orientation="horizontal",
        data=df.loc[group],
        color="experiment",
        x="value",
        slicer="gender",
        y="group",
        color_palette=["#d4f542", "#4275f5"],
    )

fig.layout.height = 800

SubplotGridFormatter(fig=fig).set_common_axis_limit(
    shared_grid_axis="rows", plot_axis="yaxis", link_axes=True
).set_common_axis_limit(
    shared_grid_axis="all", plot_axis="xaxis", link_axes=True
).set_common_coloraxis(
    shared_grid_axis="rows"
).tidy_subplots(
    title="Sex difference in response to treatment",
    col_titles=["control", "treatment"],
    row_titles=["raw data", "distribution"],
)

fig.show()

This last plot provides the following insights :
- both sexes appear to respond to treatment positively, but the response effect size looks higher in males because of a lower baseline.
- an experimental effect on the response to treatment is apparent in the scatter plot. This has to be taken into account in follow up statistical analyses.

 Full details of the API : {mod}`~statsplotly.utils`.